In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import json
import gzip
%pip install geopandas
%pip install contextily
%pip install plotly
import plotly.express as px
import folium
import geopandas as gpd
import contextily as ctx



In [ ]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)

In [ ]:
meta = [item for item in parse('meta-California.json.gz')]
meta_df = pd.DataFrame(meta)

In [ ]:
rating_df = pd.read_csv('rating-California.csv.gz', compression='gzip')

In [ ]:
rating_df

In [ ]:
meta_df

In [ ]:
meta_df['category']

In [ ]:
meta_df_restaurants = meta_df[meta_df['category'].apply(lambda categories: any('restaurant' in category.lower() for category in categories) if categories is not None else False)]

print(len(meta_df_restaurants))

In [ ]:
# Step 2: Get list of gmap_id
restaurant_ids = meta_df_restaurants['gmap_id'].tolist()

# Step 3: Filter rating_df
rating_df = rating_df[rating_df['business'].isin(restaurant_ids)]

In [ ]:
rating_df

In [ ]:
gdf = gpd.GeoDataFrame(
    meta_df, geometry=gpd.points_from_xy(meta_df.longitude, meta_df.latitude))

# Set the CRS for GeoDataFrame to EPSG:4326
gdf.set_crs(epsg=4326, inplace=True)
gdf = gdf.to_crs(epsg=3857)


In [ ]:
# Define bounds for California (in Web Mercator coordinates)
california_bounds = {
    "minx": -13884029, # Corresponding to the westernmost point
    "miny": 3832058,   # Corresponding to the southernmost point
    "maxx": -12765716, # Corresponding to the easternmost point
    "maxy": 5165920    # Corresponding to the northernmost point
}

# Create a subplot
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the points
gdf.plot(ax=ax, color='red', alpha=0.6, markersize=5)

# Add a basemap
ctx.add_basemap(ax, crs=gdf.crs, source=ctx.providers.OpenStreetMap.Mapnik)

# Zoom into California
ax.set_xlim(california_bounds['minx'], california_bounds['maxx'])
ax.set_ylim(california_bounds['miny'], california_bounds['maxy'])

ax.set_axis_off()
plt.title('Restaurant Locations in California')
plt.show()

